In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import os
from einops import rearrange, repeat
import einops
from glob import glob
from math import log
import math
import gc
import pickle

ModuleNotFoundError: No module named 'einops'

In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

2.7.1+cu118
11.8
Tesla V100-SXM2-32GB


In [3]:
# 🧩 Imports
import torch
import torch.nn as nn
from mamba_ssm.modules.mamba_simple import Mamba

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🔧 Hyperparameters
batch_size = 2
seq_len = 16
dim = 64  # model dimension

# 📦 Dummy input
x = torch.randn(batch_size, seq_len, dim).to(device)

# ⚙️ Mamba Block Configuration
mamba_block = Mamba(
    d_model=dim,
    layer_idx=0,
    # Optional configs:
    # dt_rank="auto",
    # d_state=16,
    # expand=2,
).to(device)

# 🧠 Forward pass
output = mamba_block(x)
print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 16, 64])
Output shape: torch.Size([2, 16, 64])


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
 
from mamba_ssm import Mamba
from mamba_ssm import RMSNorm
 
class MambaGPT(nn.Module):
    def __init__(self, embed_dim=512, num_layers=12, d_state=64, d_conv=4, expand=2, klen=10, dropout = 0.2):
        super().__init__()
        self.embed_dim = embed_dim
        self.embedder = Embedder(embed_dim = embed_dim, dropout = dropout)
        self.mamba_layers = nn.ModuleList(
            [nn.Sequential(RMSNorm(embed_dim), 
                           Mamba(d_model=embed_dim, d_state=d_state, d_conv=d_conv, expand=expand),
                           nn.Dropout(dropout)) 
             for _ in range(num_layers)]
        )
        self.output_layer = nn.Linear(embed_dim, klen * 3)
        self.norm = RMSNorm(embed_dim)
 
    def change_maskval(self, x, init_val = -100, target_val = 0):
        out = x.clone()
        out[out == init_val] = target_val
        return out
        
    def forward(self, x, return_z = False):
        in_scale, out_scale = 1.0, 1.0
        x = self.change_maskval(x) # for training stability
        x = self.embedder(x)  # Add slight noise
        
        x = x * in_scale
        feature = []
        # feature = 0
        for layer in self.mamba_layers:
            z = layer(x)
            feature.append(z)
            x = z + x           
            
        x = self.norm(x)
        
        if return_z:
            return self.output_layer(x) * out_scale, feature
        else:
            return self.output_layer(x) * out_scale

ImportError: cannot import name 'Embedder' from 'mamba_ssm' (/home/idies/workspace/Storage/tbalasoor/persistent/2025/Mamba/mamba/mamba_ssm/__init__.py)

In [10]:
model = MambaGPT()

NameError: name 'Embedder' is not defined